In [ ]:
!pip install streamlit bcrypt PyJWT pyngrok

In [ ]:
%%writefile config.py
import os
from datetime import timedelta

JWT_SECRET = os.getenv("JWT_SECRET", "CHANGE_THIS_SECRET")
JWT_ALGORITHM = "HS256"
TOKEN_EXPIRY_MINUTES = 60

MIN_PASSWORD_LENGTH = 8

Overwriting config.py


In [ ]:
%%writefile db.py
import sqlite3

DB_NAME = "policynav_users.db"

def get_connection():
    return sqlite3.connect(DB_NAME, check_same_thread=False)

def init_db():
    conn = get_connection()
    cursor = conn.cursor()

    cursor.execute("""
    CREATE TABLE IF NOT EXISTS users (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        username TEXT UNIQUE NOT NULL,
        email TEXT UNIQUE NOT NULL,
        password_hash TEXT NOT NULL,
        security_question TEXT NOT NULL,
        security_answer_hash TEXT NOT NULL
    )
    """)

    conn.commit()
    conn.close()

def create_user(username, email, password_hash, question, answer_hash):
    conn = get_connection()
    cursor = conn.cursor()
    try:
        cursor.execute("""
        INSERT INTO users (username, email, password_hash, security_question, security_answer_hash)
        VALUES (?, ?, ?, ?, ?)
        """, (username, email, password_hash, question, answer_hash))
        conn.commit()
        return True, "User created successfully"
    except sqlite3.IntegrityError:
        return False, "Username or Email already exists"
    finally:
        conn.close()

def get_user_by_email(email):
    conn = get_connection()
    cursor = conn.cursor()
    cursor.execute(
        "SELECT username, email, password_hash, security_question, security_answer_hash FROM users WHERE email=?",
        (email,)
    )
    user = cursor.fetchone()
    conn.close()
    return user

def update_password(email, new_hash):
    conn = get_connection()
    cursor = conn.cursor()
    cursor.execute("UPDATE users SET password_hash=? WHERE email=?", (new_hash, email))
    conn.commit()
    conn.close()

Overwriting db.py


In [122]:
%%writefile auth.py
import re
import bcrypt
import jwt
from datetime import datetime, timedelta
from config import JWT_SECRET, JWT_ALGORITHM, TOKEN_EXPIRY_MINUTES

def hash_text(text: str) -> str:
    return bcrypt.hashpw(text.encode(), bcrypt.gensalt()).decode()

def verify_text(text: str, hashed: str) -> bool:
    return bcrypt.checkpw(text.encode(), hashed.encode())

def validate_email(email: str) -> bool:
    pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
    return re.match(pattern, email) is not None

def validate_password(password: str):
    if len(password) < 8:
        return False, "Password must be at least 8 characters"
    if not re.search(r"[A-Z]", password):
        return False, "Password must contain at least 1 uppercase letter"
    if not re.search(r"[a-z]", password):
        return False, "Password must contain at least 1 lowercase letter"
    if not re.search(r"\d", password):
        return False, "Password must contain at least 1 number"
    return True, "Valid password"

def generate_token(email: str):
    payload = {
        "email": email,
        "exp": datetime.utcnow() + timedelta(minutes=TOKEN_EXPIRY_MINUTES)
    }
    return jwt.encode(payload, JWT_SECRET, algorithm=JWT_ALGORITHM)

def verify_token(token: str):
    try:
        return jwt.decode(token, JWT_SECRET, algorithms=[JWT_ALGORITHM])
    except:
        return None

Overwriting auth.py


In [133]:
%%writefile app.py
import streamlit as st
from db import *
from auth import *
import base64

init_db()

st.set_page_config(page_title="PolicyNav Auth", layout="centered")

# ---------- BACKGROUND IMAGE ----------
def set_bg(image_file):
    with open(image_file, "rb") as f:
        encoded = base64.b64encode(f.read()).decode()

    st.markdown(f"""
        <style>
        .stApp {{
            background: url("data:image/jpg;base64,{encoded}") no-repeat center center fixed;
            background-size: cover;
        }}
        </style>
    """, unsafe_allow_html=True)

set_bg("99.jpg")  # ← your wallpaper file

# ---------- SESSION STATE ----------
if "token" not in st.session_state:
    st.session_state.token = None

if "page" not in st.session_state:
    st.session_state.page = "Login"

if "reset_stage" not in st.session_state:
    st.session_state.reset_stage = 0

if "page" not in st.session_state:
    st.session_state.page = "Login"

def navigate(page):
    st.session_state.page = page

# ---------- LOGIN PAGE ----------
def login_page():
    st.title("PolicyNav – Login")

    email = st.text_input("Email")
    password = st.text_input("Password", type="password")

    if st.button("Login"):
        if not email:
            st.error("Email required"); return
        if not password:
            st.error("Password required"); return

        user = get_user_by_email(email)
        if not user:
            st.error("Email not registered"); return

        if not verify_text(password, user[2]):
            st.error("Incorrect password"); return

        st.session_state.token = generate_token(email)
        navigate("Dashboard")

    col1, col2 = st.columns(2)
    col1.button("Signup", on_click=lambda: navigate("Signup"))
    col2.button("Forgot Password", on_click=lambda: navigate("Forgot"))

# ---------- SIGNUP PAGE ----------
def signup_page():
    st.title("Signup")

    username = st.text_input("Username")
    email = st.text_input("Email")
    password = st.text_input("Password", type="password")
    confirm = st.text_input("Confirm Password", type="password")

    question = st.selectbox("Security Question", [
        "What is your favorite book?",
        "What is your dream job?"
    ])

    answer = st.text_input("Security Answer")

    if st.button("Create Account"):
        if not username:
            st.error("Username required"); return
        if not email:
            st.error("Email required"); return
        if not validate_email(email):
            st.error("Invalid email format"); return

        valid, msg = validate_password(password)
        if not valid:
            st.error(msg); return

        if password != confirm:
            st.error("Passwords do not match"); return
        if not answer:
            st.error("Security answer required"); return

        success, msg = create_user(
            username,
            email,
            hash_text(password),
            question,
            hash_text(answer)
        )

        if success:
            st.success("Account created successfully")
            navigate("Login")
        else:
            st.error(msg)

    st.button("Back to Login", on_click=lambda: navigate("Login"))

# ---------- DASHBOARD ----------
def dashboard_page():
    payload = verify_token(st.session_state.token)

    if not payload:
        st.error("Session expired. Please login again.")
        navigate("Login")
        return

    user = get_user_by_email(payload["email"])
    username = user[0]

    st.title(f"Welcome, {username} 👋")

    col1, col2 = st.columns(2)

    if col1.button("Logout"):
      st.session_state.token = None
      navigate("Login")

    if col2.button("Reset Password"):
      navigate("Reset")

# ---------- RESET ----------
def reset_page():
    st.title("🔒 Reset Password")

    payload = verify_token(st.session_state.token)

    if not payload:
        st.error("Session expired. Please login again.")
        navigate("Login")
        return

    user = get_user_by_email(payload["email"])

    old_password = st.text_input("Old Password", type="password")
    new_password = st.text_input("New Password", type="password")
    confirm_password = st.text_input("Confirm New Password", type="password")

    if st.button("Update Password"):

        if not old_password or not new_password or not confirm_password:
            st.error("All fields required")
            return

        if not verify_text(old_password, user[2]):
            st.error("Old password incorrect")
            return

        valid, msg = validate_password(new_password)
        if not valid:
            st.error(msg)
            return

        if new_password != confirm_password:
            st.error("Passwords do not match")
            return

        update_password(payload["email"], hash_text(new_password))

        st.success("Password updated successfully!")
        st.session_state.token = None
        navigate("Login")

    st.button("Back to Dashboard", on_click=lambda: navigate("Dashboard"))

def reset_to_login():
    st.session_state.reset_stage = 0
    navigate("Login")

# ---------- FORGOT PASSWORD ----------
def forgot_page():
    st.title("Forgot Password")

    # ----- Stage 0 → Ask Email -----
    if st.session_state.reset_stage == 0:

        email = st.text_input("Enter your registered email")

        if st.button("Verify Email"):
            if not email:
                st.error("Email required"); return

            user = get_user_by_email(email)
            if not user:
                st.error("Email not found"); return

            st.session_state.reset_email = email
            st.session_state.reset_question = user[3]
            st.session_state.reset_stage = 1
            st.rerun()

        st.button("Back to Login", on_click=lambda: navigate("Login"))

    # ----- Stage 1 → Security Question -----
    elif st.session_state.reset_stage == 1:

        st.info(st.session_state.reset_question)

        answer = st.text_input("Your Answer")
        new_password = st.text_input("New Password", type="password")

        if st.button("Reset Password"):

            if not answer:
                st.error("Security answer required"); return
            if not new_password:
                st.error("New password required"); return

            user = get_user_by_email(st.session_state.reset_email)

            if not verify_text(answer, user[4]):
                st.error("Incorrect security answer"); return

            valid, msg = validate_password(new_password)
            if not valid:
                st.error(msg); return

            update_password(
                st.session_state.reset_email,
                hash_text(new_password)
            )

            st.success("Password reset successful!")
            st.session_state.reset_stage = 0
            navigate("Login")
        st.button("⬅ Back", on_click=lambda: reset_to_login())

# ---------- ROUTER ----------
if st.session_state.page == "Signup":
    signup_page()
elif st.session_state.page == "Dashboard":
    dashboard_page()
elif st.session_state.page == "Forgot":
    forgot_page()
elif st.session_state.page == "Reset":
    reset_page()
else:
    login_page()

Overwriting app.py


In [135]:
!streamlit run app.py &>/content/log.txt &

In [136]:
from pyngrok import ngrok
ngrok.set_auth_token("TOKEN")
print(ngrok.connect(8501))


NgrokTunnel: "https://066c-34-11-85-163.ngrok-free.app" -> "http://localhost:8501"
